In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2

from sklearn.utils import shuffle
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
import keras_cv

d:\master\object-detection\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
input_data = tf.ones(shape=(8, 224, 224, 3))

# Randomly initialized backbone
model =keras_cv.models.MobileNetV3SmallBackbone()
output = model(input_data)

In [3]:
model.summary()

Model: "mobile_net_v3_backbone"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, None,      │          0 │ input_layer[0][0] │
│ (Rescaling)         │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv (Conv2D)       │ (None, None,      │        432 │ rescaling[0][0]   │
│                     │ None, 16)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv_BatchNorm      │ (None, None,      │         64 │ Conv[0][0]        │
│ (BatchNormalizatio… │ None, 16)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, None,      │          0 │ Conv_BatchNorm[0… │
│                     │ None, 16)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, None,      │          0 │ add[0][0]         │
│                     │ None, 16)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_1          │ (None, None,      │          0 │ re_lu[0][0]       │
│ (Multiply)          │ None, 16)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, None,      │          0 │ Conv_BatchNorm[0… │
│                     │ None, 16)         │            │ multiply_1[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, None,      │          0 │ multiply[0][0]    │
│ (ZeroPadding2D)     │ None, 16)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, None,      │        144 │ expanded_conv_de… │
│ (DepthwiseConv2D)   │ None, 16)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, None,      │         64 │ expanded_conv_de… │
│ (BatchNormalizatio… │ None, 16)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, None,      │          0 │ expanded_conv_de… │
│                     │ None, 16)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ squeeze_and_excite… │ (None, None,      │        280 │ re_lu_1[0][0]     │
│ (SqueezeAndExcite2… │ None, 16)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, None,      │        256 │ squeeze_and_exci… │
│ (Conv2D)            │ None, 16)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, None,      │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ None, 16)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_ex… │ (None, None,      │      1,152 │ expanded_conv_pr… │
│ (Conv2D)            │ None, 72)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_ex… │ (None, None,      │        288 │ expanded_conv_1_

 Total params: 939,120 (3.58 MB)

 Trainable params: 927,008 (3.54 MB)

 Non-trainable params: 12,112 (47.31 KB)

In [4]:
layer_name = 're_lu_16'
split_layer = model.get_layer(layer_name)

In [5]:
sub_model1 = tf.keras.Model(inputs = model.input, outputs = split_layer.output)
sub_model2 = tf.keras.Model(inputs = split_layer.output, outputs = model.output)

In [6]:
sub_model1.compile()

In [9]:
tf.saved_model.save(sub_model1, 'sub')

INFO:tensorflow:Assets written to: sub\assets


INFO:tensorflow:Assets written to: sub\assets


In [10]:
#model = tf.keras.models.load_model('model.h5')
converter = tf.lite.TFLiteConverter.from_saved_model('sub')
tflite_model = converter.convert()
open("conv_model.tflite", "wb").write(tflite_model)

: 

: 

In [ ]:
input_data = tf.ones(shape=(8, 224, 224, 3))

# Pretrained backbone
model = keras_cv.models.YOLOV8Backbone.from_preset(
    "yolo_v8_xs_backbone"
)
output = model(input_data)

# Randomly initialized backbone with a custom config
model = keras_cv.models.YOLOV8Backbone(
    stackwise_channels=[128, 256, 512, 1024],
    stackwise_depth=[3, 9, 9, 3],
    include_rescaling=False,
)
output = model(input_data)